In [1]:
import os
import cv2
import face_recognition
import pickle
from datetime import datetime
import numpy as np

d:\PROGRAMS\Projects\face recoginition\venv\Lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [2]:
ATTENDANCE_LOG = "attendance_log.csv"
USER_DATA_DIR = "user_data"
UNKNOWN_FACES_DIR = "unknown_faces"
THRESHOLD = 0.4

In [3]:
def load_user_data():
    user_data = {}
    if not os.path.exists(USER_DATA_DIR):
        os.makedirs(USER_DATA_DIR)
        return user_data
    
    for user_id in os.listdir(USER_DATA_DIR):
        user_dir = os.path.join(USER_DATA_DIR, user_id)
        data_file = os.path.join(user_dir, f"{user_id}_data.pkl")
        
        if os.path.exists(data_file):
            with open(data_file, 'rb') as f:
                user_data[user_id] = pickle.load(f)
    
    return user_data

In [4]:
def initialize_attendance_log():
    if not os.path.exists(ATTENDANCE_LOG):
        with open(ATTENDANCE_LOG, 'w') as f:
            f.write("timestamp,user_id,user_name,department,action\n")

In [5]:
def log_attendance(user_data, action):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(ATTENDANCE_LOG, 'a') as f:
        f.write(f"{timestamp},{user_data['user_id']},{user_data['user_name']},{user_data['department']},{action}\n")
    print(f"{action} recorded for {user_data['user_name']} ({user_data['user_id']})")

In [6]:
known_users = load_user_data()
known_encodings = []
known_ids = []
user_info = {}

for user_id, data in known_users.items():
    for encoding in data['face_encodings']:
        known_encodings.append(encoding)
        known_ids.append(user_id)
        user_info[user_id] = data

In [ ]:
def check_in_out(action):
    print("Starting face recognition for attendance...")
    print("Press 'q' to quit")
    last_recognized = None
    last_recognition_time = datetime.min

    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("Error: Could not read frame from camera")
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame)
        if len(face_locations) != 1:
            cv2.putText(frame, "Please ensure one face in frame", (30, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            cv2.imshow('Face Recognition - Press Q to quit', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            continue
        
        face_encoding = face_recognition.face_encodings(rgb_frame, face_locations)[0]
        top, right, bottom, left = face_locations[0]
        
        matches = face_recognition.compare_faces(known_encodings, face_encoding, THRESHOLD)
        face_distances = face_recognition.face_distance(known_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        
        if matches[best_match_index]:
            user_id = known_ids[best_match_index]
            user_data = user_info[user_id]
            name = user_data['user_name']
            current_time = datetime.now()
            time_since_last = (current_time - last_recognition_time).total_seconds()
            
            if user_id != last_recognized or time_since_last > 120:
                log_attendance(user_data, action)
                last_recognized = user_id
                last_recognition_time = current_time
            
            color = (0, 255, 0)
            status_text = f"Recognized: {name} ({action})"
        else:
            color = (0, 0, 255)
            status_text = "Unknown user - not registered"
            name = "Unknown"
        
        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), color, cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.8, (255, 255, 255), 1)
        cv2.putText(frame, status_text, (30, 30), font, 0.8, (255, 255, 255), 2)
        
        cv2.imshow('Face Recognition - Press Q to quit', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


In [8]:
initialize_attendance_log()

In [9]:
video_capture = cv2.VideoCapture(0)

In [10]:
check_in_out("Check in")

Starting face recognition for attendance...
Press 'q' to quit
Check in recorded for sr (1)
Check in recorded for sr (1)


In [11]:
check_in_out("Check out")

Starting face recognition for attendance...
Press 'q' to quit


In [12]:
video_capture.release()
cv2.destroyAllWindows()